## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

def calibratePinhole(chessboard_row, chessboard_col, filePaths):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chessboard_row*chessboard_col,3), np.float32)
    objp[:,:2] = np.mgrid[0:chessboard_col,0:chessboard_row].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    
    # Make a list of calibration images
    images = glob.glob(filePaths)
    
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (chessboard_col,chessboard_row),None)
        
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    
    # Calibrate
    ret_calib, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    
    # Visualization
    for fname in images:
        img = cv2.imread(fname)
        
        # Undistort
        undist = cv2.undistort(img, mtx, dist, None, mtx)
        
        # Draw and display the corners
        #undist = cv2.drawChessboardCorners(undist, (9,6), corners, ret)
        cv2.imshow('undist',undist)
        cv2.waitKey(500)
    
    cv2.destroyAllWindows()
    return mtx, dist


mtx, dist = calibratePinhole(6, 9, '../camera_cal/calibration*.jpg')

## And so on and so forth...

In [2]:
import os

class LineDetector:
    def __init__(self,mtx, dist):
        self.mtx = mtx
        self.dist = dist
        # perspective transformation
        top_left = (557,475)
        top_right = (729,475)
        bottom_left = (253,697)
        bottom_right = (1069,697)
        src = np.float32([[top_left[0],top_left[1]],[top_right[0],top_right[1]],[bottom_left[0],bottom_left[1]],[bottom_right[0],bottom_right[1]]])
        # Define conversions in x and y from pixels space to meters
        dst = np.float32([[290,431],[990,431],[290,719],[990,719]]) # y = 24 pix/m, left to right = 700 pixel
        self.M = cv2.getPerspectiveTransform(src,dst)
        self.Minv = cv2.getPerspectiveTransform(dst,src)
        
        # Choose the number of sliding windows
        self.nwindows = 15
        
        self.slided_windows_left = []
        self.slided_windows_right = []
    
    def get_valid_windows(self, windows):
        ctr = 0
        for i in range(len(windows)):
            if(windows[i].valid == True):
                ctr += 1
        return ctr
    
    def abs_sobel_calculate(self,img, orient='x', kernel_size=None):
        # Apply the following steps to img
        # 1) Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # 2) Take the derivative in x or y given orient = 'x' or 'y'
        if(orient=='x'):
            if(kernel_size==None):
                sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
            else:
                sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
        elif(orient=='y'):
            if(kernel_size==None):
                sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
            else:
                sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
        else:
            raise Exception('Unknown orientation')
        # 3) Take the absolute value of the derivative or gradient
        abs_sobel = np.absolute(sobel)
        return abs_sobel
    
    def dir_calculate(self,img, sobel_kernel=3):
        # 1) Take the gradient in x and y separately
        abs_sobelx = self.abs_sobel_calculate(img, orient='x', kernel_size=sobel_kernel)
        abs_sobely = self.abs_sobel_calculate(img, orient='y', kernel_size=sobel_kernel)
        # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
        dir_img = np.arctan2(abs_sobely, abs_sobelx)
        return dir_img
    
    def mask_threshold(self,img, thresh):
        # 5) Create a binary mask where direction thresholds are met
        binary_output = np.zeros_like(img)
        binary_output[(img >= thresh[0]) & (img <= thresh[1])] = 1
        # 6) Return this mask as your binary_output image
        return binary_output
    
    def convertTo8bit(self,abs_img,max_value):
        res = np.uint8(255*abs_img/max_value)
        return res
    
    def mag_calculate(self,img, sobel_kernel=3):
        # 2) Take the gradient in x and y separately
        abs_sobelx = self.abs_sobel_calculate(img, orient='x', kernel_size=sobel_kernel)
        abs_sobely = self.abs_sobel_calculate(img, orient='y', kernel_size=sobel_kernel)
        # 3) Calculate the magnitude 
        abs_sobel = np.sqrt(np.add(np.square(abs_sobelx), np.square(abs_sobely)))
        return abs_sobel
    
    def get_binary_warped(self,undist):
        Lab = cv2.cvtColor(undist, cv2.COLOR_BGR2Lab)
        L = Lab[:,:,0]
        B = Lab[:,:,2]
        L_masked = self.mask_threshold(L, (100, 255))
        B_masked = self.mask_threshold(B, (150, 255))
        yellow_masked = np.zeros_like(B_masked)
        yellow_masked[(L_masked == 1) & (B_masked == 1)] = 1
        yellow_masked = self.convertTo8bit(yellow_masked,1)
        yellow_warped = cv2.warpPerspective(yellow_masked, self.M, yellow_masked.shape[1::-1], flags=cv2.INTER_LINEAR)
        
        hls = cv2.cvtColor(undist, cv2.COLOR_BGR2HLS)
        S = hls[:,:,2]
        
        mag_binary = self.mag_calculate(undist, sobel_kernel=9)
        mag_binary = self.convertTo8bit(mag_binary,np.max(mag_binary))
        mag_binary = self.mask_threshold(mag_binary, (30, 100))
        mag_binary = self.convertTo8bit(mag_binary,1)
        
        S_masked = self.mask_threshold(S, (150, 255))
        S_masked = self.convertTo8bit(S_masked,1)
        warped = cv2.warpPerspective(S_masked, self.M, S_masked.shape[1::-1], flags=cv2.INTER_LINEAR)
        mag_warped = cv2.warpPerspective(mag_binary, self.M, mag_binary.shape[1::-1], flags=cv2.INTER_LINEAR)
        return warped,mag_warped,yellow_warped

    def getLanePix(self,binary_warped,slided_windows):
        lane_inds = []
        for windows in slided_windows:
            if(windows.valid == True):
                lane_inds.append(windows.nonZeroInds)
        if(len(lane_inds) == 0):
            lane_inds.append(np.array([], dtype='int64'))
        
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        lane_inds = np.unique(np.concatenate(lane_inds))
        
        x = nonzerox[lane_inds]
        y = nonzeroy[lane_inds]
        return x,y
    
    def find_lane_pixels(self,binary_warped,isLeft,init_offset_gain=1):
        # HYPERPARAMETERS
        # Set the width of the windows +/- margin
        margin = 100
        # Set height of windows - based on nwindows above and image shape
        window_height = 36
        
        # Take a histogram of the bottom of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]-(window_height*init_offset_gain):,:], axis=0)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        if (isLeft):
            x_base = np.argmax(histogram[:midpoint])
        else:
            x_base = np.argmax(histogram[midpoint:]) + midpoint
        
        initial_diff = window_height
        y_base = binary_warped.shape[0] - int(initial_diff/2)
        
        old_y = y_base + initial_diff
        
        slided_windows = []
        
        # Current positions to be updated later for each window in nwindows
        window = sliding_window(x_base,y_base,margin,window_height,binary_warped)
        
        # Step through the windows one by one
        while(self.get_valid_windows(slided_windows) < self.nwindows):
            if(window.isOutOfBound(binary_warped) == True):
                break
            
            slided_windows.append(window)
            
            ### If you found > minpix pixels, recenter next window ###
            ### (`right` or `leftx_current`) on their mean position ###
            slide_direction = np.array([0.0,window.y_center - old_y]) # x,y
            
            if (window.valid == True):
                slide_direction[0] = window.getXmean() - window.x_center
            
            # Calculate new direction in unit vector
            slide_direction = np.multiply(slide_direction,float(initial_diff)/np.linalg.norm(slide_direction))
            
            old_y = window.y_center
            
            # Apply direction vector to the current window
            window = window.createSlidedWindow(slide_direction,margin,window_height,binary_warped)
        
        # Extract left and right line pixel positions
        pixel_x,pixel_y = self.getLanePix(binary_warped,slided_windows)
    
        return pixel_x, pixel_y, slided_windows
    
    def overlay_lines(self,ori_undist,binary_warped,left_fitx,right_fitx,ploty):
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, self.Minv, (binary_warped.shape[1], binary_warped.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(ori_undist, 1, newwarp, 0.3, 0)
        return result
    
    def measure_curvature(self,left_fitx,right_fitx,ploty):
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        
        # Fit a second order polynomial to pixel positions in each fake lane line
        ##### TO-DO: Fit new polynomials to x,y in world space #####
        ##### Utilize `ym_per_pix` & `xm_per_pix` here #####
        left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
        
        # Define y-value where we want radius of curvature
        # We'll choose the maximum y-value, corresponding to the bottom of the image
        y_eval = np.max(ploty)
        
        ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
        left_curverad = (1+(2*left_fit_cr[0]*y_eval*ym_per_pix+left_fit_cr[1])**2)**(3/2)/np.abs(2*left_fit_cr[0])  ## Implement the calculation of the left line here
        right_curverad = (1+(2*right_fit_cr[0]*y_eval*ym_per_pix+right_fit_cr[1])**2)**(3/2)/np.abs(2*right_fit_cr[0])  ## Implement the calculation of the right line here
        
        return left_curverad,right_curverad
    
    def fit_polynomial(self,leftx, lefty, rightx, righty, img_height):
        ### Fit a second order polynomial to each using `np.polyfit` ###
        left_fit = np.polyfit(lefty,leftx,2)
        right_fit = np.polyfit(righty,rightx,2)
        
        # Generate x and y values for plotting
        ploty = np.linspace(0, img_height-1, img_height )
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
        
        left_curverad,right_curverad = self.measure_curvature(left_fitx,right_fitx,ploty)
        
        return left_fitx,right_fitx,ploty,left_curverad,right_curverad

In [3]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random

# Set minimum number of pixels found to recenter window
minpix = 20

class sliding_window:
    #def __init__(self):
        #self.nonZeroInds = [-1]
        #self.pix_num = 0
        #self.valid = False
    
    def __init__(self,x_center,y_center,margin,window_height,binary_warped):
        self.x_center = x_center
        self.y_center = y_center
        self.margin = margin
        self.window_height = window_height
        
        self.win_y_low = int(y_center - int(window_height/2))
        self.win_y_high = int(y_center + int(window_height/2))
        self.win_x_low = int(x_center - margin)
        self.win_x_high = int(x_center + margin)
        self.collectNonZeroInds(binary_warped)
    
    def createSlidedWindow(self,slide_direction,margin,window_height,binary_warped):
        # Apply direction vector to the current window
        x_center = self.x_center + slide_direction[0]
        y_center = self.y_center + slide_direction[1]
        return sliding_window(x_center,y_center,margin,window_height,binary_warped)
    
    def isOutOfBound(self,binary_warped):
        if((self.win_y_low < 0) or (self.win_x_low < 0) or (self.win_x_high >= binary_warped.shape[1])):
            return True
        else:
            return False
    
    def visualize(self,out_img):
        if(not self.isOutOfBound(out_img)):
            if(self.valid == True):
                colour = (0,255,0)
            else:
                colour = (0,0,255)
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(self.win_x_low,self.win_y_low),
                          (self.win_x_high,self.win_y_high),colour, 2)
        return out_img
    
    def collectNonZeroInds(self,binary_warped):
        if(self.isOutOfBound(binary_warped)):
            self.pix_num = 0
            self.valid = False
            return 0
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        self.nonzeroy = np.array(nonzero[0])
        self.nonzerox = np.array(nonzero[1])
        self.nonZeroInds = ((self.nonzeroy >= self.win_y_low) & (self.nonzeroy < self.win_y_high) & 
                            (self.nonzerox >= self.win_x_low) & (self.nonzerox < self.win_x_high)).nonzero()[0]
        self.pix_num = len(self.nonZeroInds)
        self.valid = True if (self.pix_num > minpix) else False
        return self.valid
    
    def getXmean(self):
        return np.mean(self.nonzerox[self.nonZeroInds])

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    lineDetector = LineDetector(mtx, dist)
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    binary_warped_s,binary_warped_mag,binary_warped_yel = lineDetector.get_binary_warped(undist)
    # Find our lane pixels first
    leftx, lefty, lineDetector.slided_windows_left = lineDetector.find_lane_pixels(binary_warped_yel,True)
    if(lineDetector.get_valid_windows(lineDetector.slided_windows_left) <= 2):
        leftx, lefty, lineDetector.slided_windows_left = lineDetector.find_lane_pixels(binary_warped_s,True)
        if(lineDetector.get_valid_windows(lineDetector.slided_windows_left) <= 2):
            leftx, lefty, lineDetector.slided_windows_left = lineDetector.find_lane_pixels(binary_warped_s,True,10)
            if(lineDetector.get_valid_windows(lineDetector.slided_windows_left) <= 2):
                leftx, lefty, lineDetector.slided_windows_left = lineDetector.find_lane_pixels(binary_warped_mag,True)
    
    rightx, righty, lineDetector.slided_windows_right = lineDetector.find_lane_pixels(binary_warped_yel,False)
    if(lineDetector.get_valid_windows(lineDetector.slided_windows_right) <= 2):
        rightx, righty, lineDetector.slided_windows_right = lineDetector.find_lane_pixels(binary_warped_s,False)
        if(lineDetector.get_valid_windows(lineDetector.slided_windows_right) <= 2):
            rightx, righty, lineDetector.slided_windows_right = lineDetector.find_lane_pixels(binary_warped_s,False,10)
            if(lineDetector.get_valid_windows(lineDetector.slided_windows_right) <= 2):
                rightx, righty, lineDetector.slided_windows_right = lineDetector.find_lane_pixels(binary_warped_mag,False)
    if((lineDetector.get_valid_windows(lineDetector.slided_windows_left) <= 2) or (lineDetector.get_valid_windows(lineDetector.slided_windows_right) <= 2)):
        return undist
    
    left_fitx,right_fitx,ploty,left_curverad,right_curverad = lineDetector.fit_polynomial(leftx, lefty, rightx, righty, binary_warped_s.shape[0])
    return lineDetector.overlay_lines(undist,binary_warped_s,left_fitx,right_fitx,ploty)

In [ ]:
#test
folderName = "../test_images/"
test_images = os.listdir(folderName)
for fileName in test_images:
    fileName_without = fileName.split(".")
    print(fileName)
    testOut = process_image(cv2.imread(folderName + fileName))
    cv2.imwrite("../output_images/" + fileName_without[0] + "_final." + fileName_without[1], testOut)

In [6]:
white_output = '../test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ../test_videos_output/project_video.mp4
[MoviePy] Writing video ../test_videos_output/project_video.mp4


100%|███████████████████████████████████████████████████████████████████████████▉| 1260/1261 [1:03:56<00:03,  3.04s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../test_videos_output/project_video.mp4 

Wall time: 1h 4min 4s
